# Chat

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

### Connect to LlamaCpp

In [ ]:
chat_llm = ChatOpenAI(
    base_url="http://localhost:8080/v1",
    api_key="dev",
    model="mistral",
    temperature=0,
    max_tokens=128,
)

### Invoke

In [ ]:
chat_llm.invoke([
    SystemMessage("You are an AI assistant, that helps people write pirate poems"),
    HumanMessage("What is a sentence transformer model?"),
])

### Stream

In [ ]:
response = chat_llm.stream([
    SystemMessage("You are an AI assistant, that helps people write pirate poems"),
    HumanMessage("Hello!"),
])
for token in response:
    print(token.content, end="")

### Chat with history

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at math. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | chat_llm

In [ ]:
store = {}


def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


chat_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
chat_with_history.invoke(
    {"input": "Ahoy! What's 2+2?"},
    config={"configurable": {"session_id": "123"}},
)

In [ ]:
chat_with_history.invoke(
    {"input": "Could you repeat?"},
    config={"configurable": {"session_id": "123"}},
)

In [ ]:
session_quit_token = "q"
session_restart_token = "n"
session_id = 0
print("Chat with Mistral-7B-Instruct.")
print(f"- type {session_quit_token!r} to quit")
print(f"- type {session_restart_token!r} for a new session")
print("Enter your prompt:\n")
while (query := input("> ")) != session_quit_token:
    if query == session_restart_token:
        print("Starting new session.\n")
        session_id += 1
        continue

    response = chat_with_history.stream(
        {"input": query},
        config={"configurable": {"session_id": session_id}},
    )
    for token in response:
        print(token.content, end="")